In [2]:
#Author - Quan Ta and Steven Nie
import time
import intrinio_sdk as intrinio
from intrinio_sdk.rest import ApiException
import pandas as pd
import datetime
from csv import writer

In [3]:
#API Key is replaced by "abc123" for privacy purpose
intrinio.ApiClient().set_api_key('abc123')
intrinio.ApiClient().allow_retries(True)

In [4]:
datatags = ["apturnover",
"arturnover",
"assetturnover",
"croic",
"commontocap",
"currentratio",
"ebitdagrowth",
"ebitdaqoqgrowth",
"efftaxrate",
"fcffgrowth",
"fcffqoqgrowth",
"fcfftointerestex",
"grossmargin",
"investedcapitalturnover",
"leverageratio",
"Itdebttocap",
"nnep",
"operatingmargin",
"profitmargin",
"roe_simple",
"stdebttocap",
"enterprisevalue",
"country",
"employees",
"inc_country",
"security_delisted",
"sic"]

In [5]:
datatags_metrics = ["apturnover",
"arturnover",
"assetturnover",
"croic",
"commontocap",
"currentratio",
"ebitdagrowth",
"ebitdaqoqgrowth",
"efftaxrate",
"fcffgrowth",
"fcffqoqgrowth",
"fcfftointerestex",
"grossmargin",
"investedcapitalturnover",
"leverageratio",
"Itdebttocap",
"nnep",
"operatingmargin",
"profitmargin",
"roe_simple",
"stdebttocap",
"enterprisevalue"
]

In [6]:
datatags_basics = ["country",
"employees",
"security_delisted",
"sic"            
]

In [7]:
full_column = ["apturnover",
"arturnover",
"assetturnover",
"croic",
"commontocap",
"currentratio",
"ebitdagrowth",
"ebitdaqoqgrowth",
"efftaxrate",
"fcffgrowth",
"fcffqoqgrowth",
"fcfftointerestex",
"grossmargin",
"investedcapitalturnover",
"leverageratio",
"Itdebttocap",
"nnep",
"operatingmargin",
"profitmargin",
"roe_simple",
"stdebttocap",
"enterprisevalue",
"country",
"employees",
"security_delisted",
"sic",
"ipo_price",
"current_price"]

Change the csv file name to whatever splited csv you have

In [8]:
df1 = pd.read_csv('BZ_Pro_IPOs_Calendar_No_SPAC_Unit_part2.csv')  # change name
ipo_df = df1[['Date','Symbol']][:-1]

In [9]:
for t in full_column:
    ipo_df[t] = ''

The following program will create a csv file in your folder, and it will save the result of every ticker everytime the data is collected, so if your program got interupted you dont have to restart from start and and just pick from where it got broken. remember it takes about 10 sec to collect data for each ticker, and every one of you should get a splited csv with about 850 ticker so it should take around 2.3 hrs for the program to run

And DO NOT USE COLAB for the data collection, rather run it on your local machine's jupyter

In [10]:
for symbol in ipo_df['Symbol']:
    
    print(symbol)
    
    if symbol is None:
        continue
    
    # set the definition of current day for getting current day price and for getting the ipo date for each symbol
    ipo_date = str(ipo_df.loc[ipo_df['Symbol'] == symbol, "Date"])
    #print(ipo_date)
    price_current_day = '2022-01-25'
    
    # start of block for looping though every metrics datatag
    for metric_tag in datatags_metrics:
        identifier = symbol
        tag = metric_tag
        frequency = 'yearly'
        start_date = ipo_date
        end_date = ''
        sort_order = 'asc'
        page_size = 1


        try:
            response = intrinio.CompanyApi().get_company_historical_data(identifier, tag, frequency=frequency, start_date=start_date, end_date=end_date, sort_order=sort_order, page_size=page_size) 
            tag_response = response.historical_data
            
            if len(tag_response) == 0:  # handle when the historical data is empty
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = ''

            else:
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = tag_response[0].value # write value in once there are no api exception and empty history field

        except ApiException as e: # handle other api exception
            if e.status == 401:
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = 'api_401'
            elif e.status == 403:
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = 'api_403'
            elif e.status == 404:
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = ''
            elif e.status == 429:
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = 'api_429'
            elif e.status == 500:
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = 'api_500'
            elif e.status == 503:
                ipo_df.loc[ipo_df['Symbol'] == symbol, metric_tag] = 'api_503'   
    # end of block for looping though every metrics datatag
    
    # start of block for looping through every basics datatags and collect them
    for basic_tag in datatags_basics:
        identifier = symbol
        tag = basic_tag

        try:  # no need for checking empty history data since the function directly return a string
            response_value = intrinio.CompanyApi().get_company_data_point_text(identifier, tag)
            ipo_df.loc[ipo_df['Symbol'] == symbol, basic_tag] = response_value

        except ApiException as e: # handle other api exception
            if e.status == 401:
                ipo_df.loc[ipo_df['Symbol'] == symbol, basic_tag] = 'api_401'
            elif e.status == 403:
                ipo_df.loc[ipo_df['Symbol'] == symbol, basic_tag] = 'api_403'
            elif e.status == 404:
                ipo_df.loc[ipo_df['Symbol'] == symbol, basic_tag] = ''
            elif e.status == 429:
                ipo_df.loc[ipo_df['Symbol'] == symbol, basic_tag] = 'api_429'
            elif e.status == 500:
                ipo_df.loc[ipo_df['Symbol'] == symbol, basic_tag] = 'api_500'
            elif e.status == 503:
                ipo_df.loc[ipo_df['Symbol'] == symbol, basic_tag] = 'api_503'
    
    # end of block for looping through every basics datatags and collect them
    
    # start of block for getting ipo price data
    identifier = symbol
    sort_order = 'asc'
    page_size = 1
    end_date = ''
    next_page = ''
    
    try:
        ipo_price_response = intrinio.SecurityApi().get_security_historical_data(symbol, 'adj_open_price', frequency='daily', start_date=ipo_date, end_date=end_date, sort_order=sort_order, page_size=page_size, next_page=next_page)
        ipo_price_response_hist = ipo_price_response.historical_data
        
        if len(ipo_price_response_hist) == 0: # handle when the historical data is empty
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = ''
        else:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = ipo_price_response_hist[0].value
    
    except ApiException as e: # handle other api exception
        if e.status == 401:
                ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = 'api_401'
        elif e.status == 403:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = 'api_403'
        elif e.status == 404:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = ''
        elif e.status == 429:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = 'api_429'
        elif e.status == 500:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = 'api_500'
        elif e.status == 503:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'ipo_price'] = 'api_503'
                
    # end of block for getting ipo price data
                
    # start of block for getting current price data
    identifier = symbol
    tag = 'adj_close_price'
    frequency = 'daily'
    start_date = price_current_day
    end_date = ''
    sort_order = 'asc'
    page_size = 1
    
    try:
        current_price_response = intrinio.SecurityApi().get_security_historical_data(identifier, tag, frequency=frequency, start_date=start_date, end_date=end_date, sort_order=sort_order, page_size=page_size)
        current_price_response_hist = current_price_response.historical_data
        
        if len(current_price_response_hist) == 0: # handle when the historical data is empty
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = ''
        else:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = current_price_response_hist[0].value
    
    except ApiException as e: # handle other api exception
        if e.status == 401:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = 'api_401'
        elif e.status == 403:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = 'api_403'
        elif e.status == 404:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = ''
        elif e.status == 429:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = 'api_429'
        elif e.status == 500:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = 'api_500'
        elif e.status == 503:
            ipo_df.loc[ipo_df['Symbol'] == symbol, 'current_price'] = 'api_503'
    
    # end of block for getting current price data
    
    # last part in the end will write whatever data collected about a ticker and write it into the csv file so the progress is saved
    row_list = ipo_df.loc[ipo_df['Symbol'] == symbol].values.tolist()

    with open('data_2.csv', 'a+', newline='') as write_obj:  # <- change the name of the output csv
        csv_writer = writer(write_obj)
        csv_writer.writerows(list(row_list))

DNMR
VINC
YMTX
HCAR
GOEV
MTAC
SKLZ
VIRI
UPST
WNW
GHVI
EDTX
GLAQ
FDMT
ABNB
HTOO
FLAC
MUDS
YQ
SGTX
HTPA
DCBO
TACA
FRX
GNPK
VMAR
LSAQ
SHC
CHFW
IIAC
PHIC
PIPP
INVO
INM
TSIA
TLMD
CONX
FSR
ALGM
BOAC
MAX
MACU
FHTX
YSAC
ABCM
LFTR
YGMZ
HLXA
TMPM
EAR
TMTS
CURI
IPOE
INTZ
KRON
NGAC
MAAC
IACA
ASAQ
SEAH
AVO
RCHG
CLII
YALA
ORPH
GRAY
GLSI
TSHA
BSY
FCAC
U
VTRU
RTP
AMWL
EQD
STEP
SNPR
LSPD
CRHC
INAQ
AUVI
CFII
AOUT
KYMR
FIII
STPK
FLUX
BEKE
FAII
IBEX
KSMT
AFIB
GOAC
HSAQ
PRPB
CCIV
ACEV
ITOS
ERES
PSTH
PAND
TIG
GOCO
QH
RACA
ACCD
ADTX
KCAC
AKUS
FMTX
BNR
AZEK
VRM
DADA
ZI
WPF
BMRG
GIK
LOAK
CGRO
PCPL
KROS
DFPH
PASG
NBAC
RVMD
ONEW
SDGR
ARQT
ANPC
IMAB
VVID
CLHN
RCHR
HAAC.U
GENI WS
LCYU
ENVN
MNPR
OCFT
XP
PINE
SAQN
ETNB
TELA
CNTG
FSRV
OYST
DAO
AIH
GLEO
VIR
VIE
PTON
DDOG
EXPC
NET
CFB
INMD
DT
VIST
NOVA
CSTL
OAC
MIRM
DOYU
PRMI
MORF
BSTZ
SMMC
PRVL
DRMT
CRWD
GIX
RTLR
FSLY
BEER
CPTI
JFIN
MEC
CRTX
TRVI
YJ
ACTT
ZM
PLMR
PD
RPLA/U
RUHN
LYFT
GNFT
TRNE
THCB
SHLL/U
ACAM
AVDR
VTUS
IMAC
MNCL
	HHHHU
NFE
MTC
AMTB
LEGH
SAMAU
MOGU
TC
A